In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Génération de problème d'optimisation

In [ ]:
class function:
    def __init__(self, dim, value, grad, hessian):
        self.dim = dim
        self.value = value
        self.grad = grad
        self.hessian = hessian

In [ ]:
class probleme:
    def __init__(self, f):
        self.f = f
        
    def __call__(self, x):
        return self.f.value(x)

In [ ]:
f_d = {
    "dim": 1,
    "value": lambda x: x[0]**2 - 5 * x[0] + 3,
    "grad": lambda x: np.array([2*x[0] - 5]),
    "hessian": lambda x: np.diag([2])
}
f = function(**f_d)
P = probleme(f)

In [ ]:
f_2_d = {
    "dim": 2,
    "value": lambda x: x[0]**2 + x[1]**2,
    "grad": lambda x: np.array([2*x[0], 2*x[1]]),
    "hessian": lambda x: np.diag([2, 2])
}

f_2 = function(**f_2_d)

# Méthode de Newton

In [ ]:
def backtracking(f, x, alpha=0.1, beta=0.8):
    t = 1
    desc_d = -1 * f.grad(x)
    while f.value(x + desc_d * t) > f.value(x) + alpha * t * np.dot(f.grad(x).T, desc_d):
        t = beta * t
    return t

In [ ]:
def constant(*args):
    return 0.01

In [ ]:
class Newton:
    def __init__(self, f, pas, epsilon=0.01):
        self.epsilon = epsilon
        self.f = f
        self.pas = pas
        self.save = np.array([])
        
    def __call__(self, x0):
        self.save = []
        x = x0
        self.save.append(x)
        dxN = -1 * np.dot(np.linalg.inv(self.f.hessian(x)), self.f.grad(x))
        lmd = -1 * np.dot(self.f.grad(x).T, dxN)
        while lmd / 2 > self.epsilon:
            dxN = -1 * np.dot(np.linalg.inv(self.f.hessian(x)), self.f.grad(x))
            lmd = -1 * np.dot(self.f.grad(x).T, dxN)
            t = self.pas(f, x)
            x = x + t * dxN
            self.save.append(x)
        self.save = np.array(self.save)
        return x
    
    def plot(self):
        if self.save.shape[0] == 0:
            raise Exception("The Newton method algorithm has not been run")
        if self.f.dim == 1:
            plt.figure(figsize=(15, 15))
            x = np.linspace(self.save.min() - 5, self.save.max() + 5, 1000).reshape((1, -1))
            plt.plot(x.reshape((-1)), self.f.value(x))
            plt.plot(self.save[:, 0], np.zeros((self.save.shape[0])), "rx")
            plt.show()
        elif self.f.dim == 2:
            plt.figure(figsize=(15, 15))
            x, y = np.linspace(self.save[:, 0].min(), self.save[:, 1].max(), 200), np.linspace(self.save[:, 0].min(), self.save[:, 1].max(), 200)
            X, Y = np.meshgrid(x, y)
            x_y = np.vstack([X.reshape(1, -1), Y.reshape(1, -1)]).reshape(2, -1)
            plt.contour(X, Y, self.f.value(x_y).reshape(200, -1), 15)
            plt.plot(self.save[:, 0], self.save[:, 1], "rx")
            plt.grid()
            plt.show()
        else:
            raise Exception("Dimension > 2 not implemented")

In [ ]:
meth = Newton(f, backtracking)
meth(np.array([100]))
meth.plot()